# Imports

In [ ]:
import numpy as np
import pandas as pd
import zipfile

from aif360.datasets import GermanDataset

from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover

from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing import AdversarialDebiasing

from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
from aif360.algorithms.postprocessing import RejectOptionClassification

# TF
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.AUTO_REUSE

from aif360.datasets import StandardDataset
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import utils

c:\Users\Arturo\Desktop\PhD\PublicarTFM\Code2\venv\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\Arturo\Desktop\PhD\PublicarTFM\Code2\venv\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html

# Data

In [2]:
seeds = [12345, 424242, 777, 32768, 45234]

for seed in seeds:
    print(seed)

12345
424242
777
32768
45234


In [ ]:
seed = 12345
np.random.seed(seed)

def GermanDataset1V(seed = 12345):
    """
    Read and preprocess the German dataset for the case of one sensitive variable
    (https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data).
    ====================================================================================
    Inputs:
        seed (int): seed needed to ensure reproductibility.
        
    Outputs:
        data_train (aif360.StandardDataset): Train dataset obtained from the German dataset.
        data_val (aif360.StandardDataset): Validation dataset obtained from the German dataset.
        data_test (aif360.StandardDataset): Test dataset obtained from the German dataset.
        sensitive_attribute (str): Name of the sensitive attribute .
        privileged_groups (list): list that stores a dictionary with the sensitive attribute and the privileged label.
        unprivileged_groups (list): list that stores a dictionary with the sensitive attribute and the unprivileged label.
    """

    # Read the data
    dataset_german = GermanDataset(
            protected_attribute_names=['age'],            
            privileged_classes=[lambda x: x >= 25],      
            features_to_drop=['personal_status', 'sex'] 
        )
        
    # xgboost requires labels to start at zero
    dataset_german.labels[dataset_german.labels.ravel() == 2] =  dataset_german.labels[dataset_german.labels.ravel() == 2] - 2
    dataset_german.unfavorable_label = dataset_german.unfavorable_label - 2

    # train, val, test split
    data_train, vt = dataset_german.split([0.7], shuffle=True, seed=seed)
    data_val, data_test = vt.split([0.5], shuffle=True, seed=seed)

    # We obtain sensitive attribute
    sensitive_attribute = dataset_german.protected_attribute_names[0] # age
    privileged_groups, unprivileged_groups = utils.get_privileged_groups(dataset_german)
    return data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups




def GermanDataset2V(seed = 12345, operation = "OR"):
    """
    Read and preprocess the German dataset for the case of two sensitive variables
    (https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data).
    ====================================================================================
    Inputs:
        seed (int): seed needed to ensure reproductibility.
        operation (str): bitwise operation that we apply to the sensitive variables.
                         Allowed values: "OR", "AND", "XOR".
        
    Outputs:
        data_train (aif360.StandardDataset): Train dataset obtained from the German dataset with a bitwise operation applied to two sensitive variables.
        data_val (aif360.StandardDataset): Validation dataset obtained from the German dataset with a bitwise operation applied to two sensitive variables.
        data_test (aif360.StandardDataset): Test dataset obtained from the German dataset with a bitwise operation applied to two sensitive variables.
        sensitive_attribute (str): Name of the sensitive attribute .
        privileged_groups (list): list that stores a dictionary with the sensitive attribute and the privileged label.
        unprivileged_groups (list): list that stores a dictionary with the sensitive attribute and the unprivileged label.
        data_val_single (aif360.StandardDataset): Validation dataset with just one sensitive variable.
        data_test_single (aif360.StandardDataset): Test dataset with just one sensitive variable.
    """

    # Read the data
    dataset = GermanDataset(
        protected_attribute_names=['age'],            
        privileged_classes=[lambda x: x >= 25],      
        features_to_drop=['personal_status', 'sex'] 
    )

    # load the german dataset and update the data with the OR sum of sex and age
    dataset_german_upd = utils.update_german_dataset_from_multiple_protected_attributes(dataset, operation)

    # change favorable/unfavorable labels to 1: good; 0: bad
    dataset_german_upd.labels[dataset_german_upd.labels.ravel() == 2] =  dataset_german_upd.labels[dataset_german_upd.labels.ravel() == 2] - 2
    dataset_german_upd.unfavorable_label = dataset_german_upd.unfavorable_label - 2

    # For the single dataset as well
    dataset.labels[dataset.labels.ravel() == 2] =  dataset.labels[dataset.labels.ravel() == 2] - 2
    dataset.unfavorable_label = dataset.unfavorable_label - 2

    # Train, val, test split
    data_train, vt = dataset_german_upd.split([0.7], shuffle=True, seed=seed)
    data_val, data_test = vt.split([0.5], shuffle=True, seed=seed)

    # We do the same on the single variable dataset
    _, vt = dataset.split([0.7], shuffle=True, seed=seed)
    data_val_single, data_test_single = vt.split([0.5], shuffle=True, seed=seed)

    # Obtain sensitive attributes and privileged groups
    sensitive_attribute = dataset_german_upd.protected_attribute_names[0] 
    privileged_groups, unprivileged_groups = utils.get_privileged_groups(dataset_german_upd)
    return data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups, data_val_single, data_test_single




def Homecredit1V(seed = 12345):
    """
    Read and preprocess the Homecredit dataset for the case of one sensitive variable
    (https://www.kaggle.com/c/home-credit-default-risk).
    ====================================================================================
    Inputs:
        seed (int): seed needed to ensure reproductibility.
        
    Outputs:
        data_train (aif360.StandardDataset): Train dataset obtained from the Homecredit dataset.
        data_val (aif360.StandardDataset): Validation dataset obtained from the Homecredit dataset.
        data_test (aif360.StandardDataset): Test dataset obtained from the Homecredit dataset.
        sensitive_attribute (str): Name of the sensitive attribute .
        privileged_groups (list): list that stores a dictionary with the sensitive attribute and the privileged label.
        unprivileged_groups (list): list that stores a dictionary with the sensitive attribute and the unprivileged label.
    """

    # Read the data
    dataset_homecredit = pd.read_csv('data/homecredit.zip', compression='zip', header=0, sep=',', quotechar='"')

    dataset_homecredit = utils.preprocess_homecredit(dataset_homecredit)

    dataset_homecredit_aif = utils.convert_to_standard_dataset(
            df=dataset_homecredit,
            target_label_name='TARGET',
            sensitive_attribute='AGE',
            priviledged_classes=[lambda x: x >= 25],
            favorable_target_label=[1],
            features_to_keep=[],
            categorical_features=[])

    # We make a subsample
    sample_size_hc = 5000
    dataset_homecredit_aif = dataset_homecredit_aif.subset(np.random.randint(0, 307511+1, size=(sample_size_hc)))
    # train, val, test split
    data_train, vt = dataset_homecredit_aif.split([0.7], shuffle=True, seed=seed)
    data_val, data_test = vt.split([0.5], shuffle=True, seed=seed)

    # Obtain sensitive attributes and privileged groups
    sensitive_attribute = dataset_homecredit_aif.protected_attribute_names[0] # age
    privileged_groups, unprivileged_groups = utils.get_privileged_groups(dataset_homecredit_aif)
    return data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups



def Homecredit2V(seed = 12345, operation = "OR"):
    """
    Read and preprocess the Homecredit dataset for the case of one sensitive variable
    (https://www.kaggle.com/c/home-credit-default-risk).
    ====================================================================================
    Inputs:
        seed (int): seed needed to ensure reproductibility.
        operation (str): bitwise operation that we apply to the sensitive variables.
                         Allowed values: "OR", "AND", "XOR".
        
    Outputs:
        data_train (aif360.StandardDataset): Train dataset obtained from the Homecredit dataset with a bitwise operation applied to two sensitive variables.
        data_val (aif360.StandardDataset): Validation dataset obtained from the Homecredit dataset with a bitwise operation applied to two sensitive variables.
        data_test (aif360.StandardDataset): Test dataset obtained from the Homecredit dataset with a bitwise operation applied to two sensitive variables.
        sensitive_attribute (str): Name of the sensitive attribute.
        privileged_groups (list): list that stores a dictionary with the sensitive attribute and the privileged label.
        unprivileged_groups (list): list that stores a dictionary with the sensitive attribute and the unprivileged label.
        data_val_single (aif360.StandardDataset): Validation dataset with just one sensitive variable.
        data_test_single (aif360.StandardDataset): Test dataset with just one sensitive variable.
    """

    # Load the data
    homecredit = pd.read_csv('data/homecredit.zip', compression='zip', header=0, sep=',', quotechar='"')
    homecredit = utils.preprocess_homecredit_mult(homecredit, operation = operation)
    homecredit_single = homecredit.copy(deep = True)
    
    # Transform both datasets to aif360 format
    homecredit = utils.convert_to_standard_dataset(
            df=homecredit,
            target_label_name='TARGET',
            sensitive_attribute=['PROT_ATTR'],
            priviledged_classes=[lambda x: x == 1],
            favorable_target_label=[1],
            features_to_keep=[],
            categorical_features=[])

    homecredit_single = utils.convert_to_standard_dataset(
            df=homecredit_single,
            target_label_name='TARGET',
            sensitive_attribute=['AGE'],
            priviledged_classes=[lambda x: x >= 25],
            favorable_target_label=[1],
            features_to_keep=[],
            categorical_features=[])

    # We make a subsample
    sample_size_hc = 5000
    ssample = np.random.randint(0, 307511+1, size=(sample_size_hc))
    homecredit = homecredit.subset(ssample)
    homecredit_single = homecredit_single.subset(ssample)

    # train, val, test split
    data_train, vt = homecredit.split([0.7], shuffle=True, seed=seed)
    data_val, data_test = vt.split([0.5], shuffle=True, seed=seed)

    _, vt_single = homecredit.split([0.7], shuffle=True, seed=seed)
    data_val_single, data_test_single = vt_single.split([0.5], shuffle=True, seed=seed)

    # Obtain sensitive attributes and privileged groups
    sensitive_attribute = homecredit.protected_attribute_names[0] 
    privileged_groups, unprivileged_groups = utils.get_privileged_groups(homecredit)

    return data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups, data_val_single, data_test_single


def simul1V(seed = 12345, N = 1000, p1 = 0.5, p2 = 0.5):
    """
    Obtain a simulated dataset from the toy model for the case of one sensitive variable
    ====================================================================================
    Inputs:
        seed (int): seed needed to ensure reproductibility.
        N (int): number of individuals in the dataset.
        p1 (float, between 0.0 and 1.0): probability for a binomial distribution from which to draw the first sensitive variable.
        p2 (float, between 0.0 and 1.0): probability for a binomial distribution from which to draw the second sensitive variable.
        
    Outputs:
        data_train (aif360.StandardDataset): Train dataset obtained from the simulation.
        data_val (aif360.StandardDataset): Validation dataset obtained from the simulation.
        data_test (aif360.StandardDataset): Test dataset obtained from the simulation.
        sensitive_attribute (str): Name of the sensitive attribute .
        privileged_groups (list): list that stores a dictionary with the sensitive attribute and the privileged label.
        unprivileged_groups (list): list that stores a dictionary with the sensitive attribute and the unprivileged label.
    """

    # Create variables
    vars = dict()

    # Sensitive variables (drawn from a binomial distribution)
    vars['sens1'] = np.random.binomial(n = 1, p = p1, size = N)
    vars['sens2'] = np.random.binomial(n = 1, p = p2, size = N)

    # v1, v2 (noisy measurements of the sensitive variables) and their sum
    vars['v1'] = np.random.normal(loc = vars['sens1'], scale = 1.0, size = N)
    vars['v2'] = np.random.normal(loc = vars['sens2'], scale = 1.0, size = N)
    vars['sum'] = vars['v1'] + vars['v2']

    # Noisy measurements of the sum of v1 and v2
    vars['indirect'] = np.random.normal(loc = vars['sum'], scale = 1.0, size = N)
    vars['weight_response'] = np.random.normal(loc = vars['sum'], scale = 1.0, size = N)

    # Response variable
    vars['response'] = vars['weight_response'] > 0.0

    # Create the dataset with the correct variables
    final_vars = ['sens1', 'sens2', 'indirect', 'response']
    df = dict()
    for name in final_vars:
        df[name] = vars[name]
    
    # Transform the sensitive variables to boolean
    df['sens1'] = df['sens1'] == 1
    df['sens2'] = df['sens2'] == 1

    # Create the dataset from the dictionary
    df = pd.DataFrame(df)

    # Convert to standard dataset
    data = utils.convert_to_standard_dataset(
        df=df,
        target_label_name = 'response',
        sensitive_attribute = ['sens1'],
        priviledged_classes = [lambda x: x == 1],
        favorable_target_label = [1],
        features_to_keep = [],
        categorical_features = []
    )

    # train, val, test split
    data_train, vt = data.split([0.7], shuffle=True, seed=seed)
    data_val, data_test = vt.split([0.5], shuffle=True, seed=seed)

    # Obtain sensitive attributes and privileged groups
    sensitive_attribute = data.protected_attribute_names[0] 
    privileged_groups, unprivileged_groups = utils.get_privileged_groups(data)

    return data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups


def simul2V(seed = 12345, operation = "OR", N = 1000, p1 = 0.5, p2 = 0.5):
    """
    Obtain a simulated dataset from the toy model for the case of two sensitive variables
    =====================================================================================
    Inputs:
        seed (int): seed needed to ensure reproductibility.
        operation (str): bitwise operation that we apply to the sensitive variables.
                         Allowed values: "OR", "AND", "XOR".
        N (int): number of individuals in the dataset.
        p1 (float, between 0.0 and 1.0): probability for a binomial distribution from which to draw the first sensitive variable.
        p2 (float, between 0.0 and 1.0): probability for a binomial distribution from which to draw the second sensitive variable.
        
    Outputs:
        data_train (aif360.StandardDataset): Train dataset obtained from the simulation with a bitwise operation applied to two sensitive variables.
        data_val (aif360.StandardDataset): Validation dataset obtained from the simulation with a bitwise operation applied to two sensitive variables.
        data_test (aif360.StandardDataset): Test dataset obtained from the simulation with a bitwise operation applied to two sensitive variables.
        sensitive_attribute (str): Name of the sensitive attribute.
        privileged_groups (list): list that stores a dictionary with the sensitive attribute and the privileged label.
        unprivileged_groups (list): list that stores a dictionary with the sensitive attribute and the unprivileged label.
        data_val_single (aif360.StandardDataset): Validation dataset with just one sensitive variable.
        data_test_single (aif360.StandardDataset): Test dataset with just one sensitive variable.
    """

    # Create variables
    vars = dict()

    # Sensitive variables (drawn from a binomial distribution)
    vars['sens1'] = np.random.binomial(n = 1, p = p1, size = N)
    vars['sens2'] = np.random.binomial(n = 1, p = p2, size = N)

    # v1, v2 (noisy measurements of the sensitive variables) and their sum
    vars['v1'] = np.random.normal(loc = vars['sens1'], scale = 1.0, size = N)
    vars['v2'] = np.random.normal(loc = vars['sens2'], scale = 1.0, size = N)
    vars['sum'] = vars['v1'] + vars['v2']

    # Noisy measurements of the sum of v1 and v2
    vars['indirect'] = np.random.normal(loc = vars['sum'], scale = 1.0, size = N)
    vars['weight_response'] = np.random.normal(loc = vars['sum'], scale = 1.0, size = N)

    # Response variable
    vars['response'] = vars['weight_response'] > 0.0

    # Create the dataset with the correct variables
    final_vars = ['sens1', 'sens2', 'indirect', 'response']
    df = dict()
    for name in final_vars:
        df[name] = vars[name]
    
    df['sens1'] = df['sens1'] == 1
    df['sens2'] = df['sens2'] == 1

    # Apply bitwise operation
    if operation == 'OR':
        df['prot_attr'] = np.logical_or(df['sens1'], df['sens2'])

    elif operation == 'AND':
        df['prot_attr'] = np.logical_and(df['sens1'], df['sens2'])

    elif operation == 'XOR':
        df['prot_attr'] = np.logical_xor(df['sens1'], df['sens2'])

    df = pd.DataFrame(df)

    # Convert to standard datasets
    data_single = utils.convert_to_standard_dataset(
        df=df,
        target_label_name = 'response',
        sensitive_attribute = ['sens1'],
        priviledged_classes = [lambda x: x == 1],
        favorable_target_label = [1],
        features_to_keep = [],
        categorical_features = []
    )

    data = utils.convert_to_standard_dataset(
        df=df,
        target_label_name = 'response',
        sensitive_attribute = ['prot_attr'],
        priviledged_classes = [lambda x: x == 1],
        favorable_target_label = [1],
        features_to_keep = [],
        categorical_features = []
    )

    # train, val, test split
    data_train, vt = data.split([0.7], shuffle=True, seed=seed)
    data_val, data_test = vt.split([0.5], shuffle=True, seed=seed)

    _, vt_single = data_single.split([0.7], shuffle=True, seed=seed)
    data_val_single, data_test_single = vt_single.split([0.5], shuffle=True, seed=seed)

    # Obtain sensitive attributes and privileged groups
    sensitive_attribute = data.protected_attribute_names[0] 
    privileged_groups, unprivileged_groups = utils.get_privileged_groups(data)

    return data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups, data_val_single, data_test_single


# Variables

In [4]:
# Inicializamos diccionarios
methods = dict()

# RRange of thresholds
thresh_sweep = np.linspace(0.01, 1.0, 50)

metrics_sweep = dict()

# Store results from validation and test
metrics_best_thresh_validate = dict()
metrics_best_thresh_test = dict()

In [5]:
modelsNames = [
    'logreg',
    'xgboost'
#    'adversarial',
#    'metafair',
#    'piremover'
]

modelsTrain = {
    'logreg': LogisticRegression,
    'xgboost': XGBClassifier
#    'adversarial': AdversarialDebiasing,
#    'metafair': MetaFairClassifier,
#    'piremover': PrejudiceRemover
}

modelsArgs = {
    'logreg': {
        'solver': 'liblinear',
        'random_state': seed
    },
    'xgboost': {
        'eval_metric': 'error',
        'eta':0.1,
        'max_depth':6,
        'subsample':0.8
    }
#    'adversarial': {
#        'privileged_groups': privileged_groups,
#        'unprivileged_groups': unprivileged_groups,
#        'scope_name': 'debiased_classifier',
#        'debias': True,
#        'sess': tf.session(), # Mirar esto de la sesion
#        'num_epochs': 80
#    },
#    'metafair_sr': {
#        'tau': 0.8,
#        'sensitive_attribute': sensitive_attribute,
#        'type': 'sr',
#        'seed': seed
#    },
#    'metafair_fdr': {
#        'tau': 0.8,
#        'sensitive_attribute': sensitive_attribute,
#        'type': 'fdr',
#        'seed': seed
#    },
#    'pir': {
#        'sensitive_attr': sensitive_attribute,
#        'eta': 50.0
#    }
}


# Auxiliary functions

In [6]:
def get_privileged_groups(dataset, sens_attr_ix=0):
    """
    Helper function to privileged and unprivileged group dictionaries from `dataset`
    Args:
        dataset (StandardDataset): dataset in the aif360 format.
        sens_attr_ix (int, optional): Index of the dataset.privileged_protected_attributes pointing to the sensitive attribute. Defaults to 0.
    Returns:
        tuple(list, list): privileged group and unprivileged group
    """

    sens_attr = dataset.protected_attribute_names[sens_attr_ix]
    priviledged_groups = [{sens_attr: v} for v in dataset.privileged_protected_attributes[sens_attr_ix]]
    unpriviledged_groups = [{sens_attr: v} for v in dataset.unprivileged_protected_attributes[sens_attr_ix]]

    return priviledged_groups, unpriviledged_groups



def results(val, test, method):
    # Global variables
    global methods
    global metrics_sweep
    global metrics_best_thresh_validate
    global metrics_best_thresh_test

    # Evaluate the model in a range of thresholds
    metrics_sweep[method] = utils.metrics_threshold_sweep(
        dataset=val,
        model=methods[method],
        thresh_arr=thresh_sweep
    )

    # Evaluate the metrics for the best threshold
    metrics_best_thresh_validate[method] = utils.describe_metrics(
        metrics_sweep[method],
        measurement,
        combination
        )

    # Compute the metrics in test using the best threshold for validation
    metrics_best_thresh_test[method] = utils.compute_metrics(
        dataset=test, 
        model=methods[method], 
        threshold=metrics_best_thresh_validate[method]['best_threshold'])
    


def results_mult(val, val_single, test, test_single, method):
    # Global variables
    global methods
    global metrics_sweep
    global metrics_best_thresh_validate
    global metrics_best_thresh_test

    # Evaluate the model in a range of thresholds
    metrics_sweep[method] = utils.metrics_threshold_sweep_mult(
        dataset = val,
        dataset_single = val_single,
        model = methods[method],
        thresh_arr = thresh_sweep
    )

    # Evaluate the metrics for the best threshold
    metrics_best_thresh_validate[method] = utils.describe_metrics(metrics_sweep[method])

    # Compute the metrics in test using the best threshold for validation
    metrics_best_thresh_test[method] = utils.compute_metrics_mult(
        dataset = test, 
        dataset_single = test_single,
        model = methods[method], 
        threshold = metrics_best_thresh_validate[method]['best_threshold'])
    



def resultsPost(val, val_preds, test, test_preds, model_name):
    # Global variables
    global methods
    global metrics_sweep
    global metrics_best_thresh_validate
    global metrics_best_thresh_test
    
    # Compute the metrics on the validation set
    metrics_best_thresh_validate[model_name] = utils.compute_metrics_postprocessing(
        dataset_true=val, 
        dataset_preds=val_preds, 
        model=methods[model_name], 
        required_threshold=False)
    
    # Compute the metrics on the test set
    metrics_best_thresh_test[model_name] = utils.compute_metrics_postprocessing(
        dataset_true=test, 
        dataset_preds=test_preds, 
        model=methods[model_name], 
        required_threshold=False)
    


# Benchmarks

In [7]:
def BenchmarkLogistic(data_train, data_val, data_test):
    # Global variables
    global methods
    global nvar

    # Assign the correct name
    model_name = 'logreg'

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # Model parameters
    fit_params = {'sample_weight': train.instance_weights}

    # Introduce the model in the model dict
    methods[model_name] = LogisticRegression(
        solver='liblinear',
        random_state=seed
    )

    # Train the model
    methods[model_name] = methods[model_name].fit(train.features, train.labels.ravel(), **fit_params)

    # Obtain results
    if nvar == 1:
        results(val, test, model_name)

    elif nvar == 2:
        global data_val_single
        global data_test_single
        val_single, test_single = data_val_single.copy(deepcopy = True), data_test_single.copy(deepcopy = True)
        results_mult(val, val_single, test, test_single, model_name)




def BenchmarkXGB(data_train, data_val, data_test):
    # Global variables
    global methods
    global nvar

    # Assign the correct name
    model_name = 'xgboost'

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # Model parameters
    fit_params = {'eval_metric': 'error', 'eta':0.1, 'max_depth':6, 'subsample':0.8}

    # Assign the correct dict
    methods[model_name] = XGBClassifier(**fit_params)

    # Train the model
    methods[model_name] = methods[model_name].fit(train.features, train.labels.ravel())

    # Obtain results
    if nvar == 1:
        results(val, test, model_name)

    elif nvar == 2:
        global data_val_single
        global data_test_single
        val_single, test_single = data_val_single.copy(deepcopy = True), data_test_single.copy(deepcopy = True)
        results_mult(val, val_single, test, test_single, model_name)

# Pre Processing

In [8]:
def PreprocRW(data_train, data_val, data_test, privileged_groups, unprivileged_groups, model, do_results = True):
    #Global variables
    global methods
    
    # Assign the correct name
    method = "RW"
    model_name = method + "_" + model

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)
    
    # Call the processor
    PreProcessor = Reweighing(
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )

    # Transform the data
    PreProcessor.fit(train)
    trainRW = PreProcessor.transform(train)
    valRW = PreProcessor.transform(test)
    testRW = PreProcessor.transform(val)

    # Train the model
    if model == 'adversarial':
        tf.compat.v1.reset_default_graph()
        modelsArgs[model]['sess'] = tf.Session()

    Algorithm = modelsTrain[model](**modelsArgs[model])

    if model in modelsBenchmark:
        if model == 'logreg':
            fit_params = {'sample_weight': trainRW.instance_weights}
            methods[model_name] = Algorithm.fit(trainRW.features, trainRW.labels.ravel(), **fit_params)
        else:
            methods[model_name] = Algorithm.fit(trainRW.features, trainRW.labels.ravel())
    else:
        methods[model_name] = Algorithm.fit(trainRW)
            
    # Obtain results
    if do_results:
        if nvar == 1:
            results(valRW, testRW, model_name)

        elif nvar == 2:
            global data_val_single
            global data_test_single
            val_single, test_single = data_val_single.copy(deepcopy = True), data_test_single.copy(deepcopy = True)
            results_mult(valRW, val_single, testRW, test_single, model_name)

    if model == 'adversarial':
        modelsArgs[model]['sess'].close()




def PreprocDI(data_train, data_val, data_test, sensitive_attribute, repair_level, model, do_results = True):
    #Global variables
    global methods
    
    # Assign the correct name
    method = "DI"
    model_name = method + "_" + model

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # Initialize the processor
    PreProcessor = DisparateImpactRemover(
        repair_level=repair_level,
        sensitive_attribute=sensitive_attribute
    )
    # Transform the data
    PreProcessor.fit_transform(train)
    trainDI = PreProcessor.fit_transform(train)
    valDI = PreProcessor.fit_transform(val)
    testDI = PreProcessor.fit_transform(test)

    # Train the model
    if model == 'adversarial':
        tf.compat.v1.reset_default_graph()
        modelsArgs[model]['sess'] = tf.Session()

    Algorithm = modelsTrain[model](**modelsArgs[model])

    if model in modelsBenchmark:
        if model == 'logreg':
            fit_params = {'sample_weight': trainDI.instance_weights}
            methods[model_name] = Algorithm.fit(trainDI.features, trainDI.labels.ravel(), **fit_params)
        else:
            methods[model_name] = Algorithm.fit(trainDI.features, trainDI.labels.ravel())
    else:
        methods[model_name] = Algorithm.fit(trainDI)

    # Obtain results
    if do_results:
        if nvar == 1:
            results(valDI, testDI, model_name)

        elif nvar == 2:
            global data_val_single
            global data_test_single
            val_single, test_single = data_val_single.copy(deepcopy = True), data_test_single.copy(deepcopy = True)
            results_mult(valDI, val_single, testDI, test_single, model_name)

    if model == 'adversarial':
        modelsArgs[model]['sess'].close()

# In processing

In [9]:
def InprocMeta(data_train, data_val, data_test, sensitive_attribute, quality,  tau = 0.8, do_results = True):
    # Global variables
    global methods

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # assign the correct name
    model_name = "metafair"
    model_name_quality = '{}_{}'.format(model_name, quality)

    # Initialize the model and store it in the dictionary
    methods[model_name_quality] = MetaFairClassifier(
        tau=tau,
        sensitive_attr=sensitive_attribute,
        type=quality,
        seed=seed
        )

    # Train the model
    methods[model_name_quality] = methods[model_name_quality].fit(train)

    # Obtain scores
    methods[model_name_quality].scores_train = methods[model_name_quality].predict(train).scores
    methods[model_name_quality].scores_val = methods[model_name_quality].predict(val).scores
    methods[model_name_quality].scores_test = methods[model_name_quality].predict(test).scores

    # Obtain results
    if do_results:
        if nvar == 1:
            results(val, test, model_name_quality)

        elif nvar == 2:
            global data_val_single
            global data_test_single
            val_single, test_single = data_val_single.copy(deepcopy = True), data_test_single.copy(deepcopy = True)
            results_mult(val, val_single, test, test_single, model_name_quality)



def InprocPI(data_train, data_val, data_test, sensitive_attribute, eta = 50.0, do_results = True):
    # Global variables
    global methods

    # Assign the correct name
    model_name = 'pir'
    
    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)
    
    # Initialize the model and store it in the dictionary
    methods[model_name] = PrejudiceRemover(
        sensitive_attr=sensitive_attribute,
        eta=eta
        )
    
    # Train the model
    methods[model_name] = methods[model_name].fit(train)
    
    # Obtain scores
    methods[model_name].scores_train = methods[model_name].predict(train).scores
    methods[model_name].scores_val = methods[model_name].predict(val).scores
    methods[model_name].scores_test = methods[model_name].predict(test).scores

    # Obtain results
    if do_results:
        results(val, test, model_name)



def InprocAdvs(data_train, data_val, data_test, privileged_groups, unprivileged_groups, do_results = True):
    # Global variables
    global methods
    global sess
    
    # Assign the correct name
    model_name = 'adversarial'
    
    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)
    
    #We train the model
    methods[model_name] = AdversarialDebiasing(
        privileged_groups = privileged_groups,
        unprivileged_groups = unprivileged_groups,
        scope_name = 'debiased_classifier',
        debias=True,
        sess=sess,
        num_epochs=80
    )    
    methods[model_name].fit(train)

    # Obtain results
    if do_results:
        results(val, test, model_name)

# Post-processing

In [10]:
def PosprocPlatt(data_train, data_val, data_test, privileged_groups, model_name):
    # Global variables
    global methods
    global metrics_sweep
    global metrics_best_thresh_validate
    global metrics_best_thresh_test
    
    # Assign the correct name
    fairness_method = '_Platt'

    # Validation
    #---------------

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy = True), data_val.copy(deepcopy = True), data_test.copy(deepcopy = True)

    # Copy the predictions
    model_thresh = metrics_best_thresh_validate[model_name]['best_threshold']
    val_preds = utils.update_dataset_from_model(val, methods[model_name], class_thresh = model_thresh)

    ## Platt Scaling:
    #---------------
    #1. Split training data on sensitive attribute
    val_preds_priv, val_preds_unpriv, priv_indices, unpriv_indices = utils.split_dataset_on_sensitive_attribute(
        dataset = val_preds,
        privileged_group_label = list((privileged_groups[0].values()))[0]
    )
    
    #2. Copy validation data predictions
    val_preds2 = val_preds.copy(deepcopy = True)
    
    #3. Make one model for each group
    sensitive_groups_data = {'priv': [val_preds_priv, priv_indices],
                             'unpriv': [val_preds_unpriv, unpriv_indices]}
    for group, data_group_list in sensitive_groups_data.items():
        # Assign the correct name
        model_name_group = '{}_{}_{}'.format(model_name, fairness_method, group)
        # Initialize the model, store it in the dict
        methods[model_name_group] = LogisticRegression()
        # Train the model using the validation data divided by group
        methods[ model_name_group ] = methods[model_name_group].fit(
            data_group_list[0].scores,   # data_group_list[0] -> data_val_preds_priv or data_val_preds_unpriv
            val.subset(data_group_list[1]).labels.ravel()
        ) # data_group_list[1] -> priv_indices or unpriv_indices

        # predict group probabilities, store in val_preds2
        # Platt scores are given by the predictions of the posterior probabilities
        scores_group = methods[model_name_group].predict_proba(data_group_list[0].scores)
        pos_ind_group = np.where(methods[model_name_group].classes_ == data_group_list[0].favorable_label)[0][0]
        val_preds2.scores[data_group_list[1]] = scores_group[:, pos_ind_group].reshape(-1,1)
   
    # Evaluate the model in a range of values
    thresh_sweep_platt = np.linspace(np.min(val_preds2.scores.ravel()),
                                     np.max(val_preds2.scores.ravel()),
                                     50)

    # Obtain the metrics for the val set
    metrics_sweep[model_name+fairness_method] = utils.metrics_postprocessing_threshold_sweep_from_scores(
            dataset_true = val,
            dataset_preds = val_preds,
            thresh_arr = thresh_sweep_platt
        )

    # Evaluate metrics and obtain the best thresh
    metrics_best_thresh_validate[model_name+fairness_method] = utils.describe_metrics(metrics_sweep[model_name+fairness_method])

    # Test
    #---------------

    model_thresh = metrics_best_thresh_validate[model_name]['best_threshold']
    test_preds = utils.update_dataset_from_model(test, methods[model_name], class_thresh = model_thresh)

    ## Plat Scaling:
    #---------------
    
    # 1. Divide test set using sensitive varaible's groups
    test_preds_priv, test_preds_unpriv, priv_indices, unpriv_indices = utils.split_dataset_on_sensitive_attribute(
        dataset = test_preds,
        privileged_group_label = list((privileged_groups[0].values()))[0]
    )
    # 2. Copy test data
    test_preds2 = test_preds.copy(deepcopy = True)
    
    # 3. Predict for each group
    sensitive_groups_data_test = {'priv': [test_preds_priv, priv_indices],
                                  'unpriv': [test_preds_unpriv, unpriv_indices]}

    for group, data_group_list in sensitive_groups_data_test.items():    
        # We assign the correct name
        model_name_group = '{}_{}_{}'.format(model_name, fairness_method, group)

        # Predict in each group, store the result in data_val_preds2
        # The probabilities are the Platt scores
        scores_group = methods[model_name_group].predict_proba(data_group_list[0].scores)
        pos_ind_group = np.where(methods[model_name_group].classes_ == data_group_list[0].favorable_label)[0][0]
        test_preds2.scores[data_group_list[1]] = scores_group[:, pos_ind_group].reshape(-1,1)
    
    # Obtain metrics
    metrics_best_thresh_test[model_name+fairness_method] = utils.compute_metrics_from_scores(
        dataset_true = test,
        dataset_pred = test_preds2,
        threshold = metrics_best_thresh_validate[model_name+fairness_method]['best_threshold']
    )

     




def PosprocEqoddsLABELS(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model_name):
    # Global variables
    global methods
    global metrics_best_thresh_validate    
    
    # Assign the correct name
    fairness_method = '_eqOdds' 

    # Copy the dataset
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # Copy the predictions of the base model
    train_preds = utils.update_dataset_from_model(train, methods[model_name])
    val_preds = utils.update_dataset_from_model(val, methods[model_name])
    test_preds = utils.update_dataset_from_model(test, methods[model_name])

    # Initialize the model and store the predictions
    methods[model_name+fairness_method] = EqOddsPostprocessing(
        privileged_groups = privileged_groups,
        unprivileged_groups = unprivileged_groups, 
        seed = seed)

    # Train the model
    methods[model_name+fairness_method] = methods[model_name+fairness_method].fit(train, train_preds)

    # Evaluate the model in a range of thresholds
    metrics_sweep[model_name+fairness_method] = utils.metrics_postprocessing_threshold_sweep(
        dataset_true=val,
        dataset_preds=val_preds,
        model=methods[model_name+fairness_method],
        thresh_arr=thresh_sweep,
        scores_or_labels='labels'
    )

    # Evaluate the model for the best threshold
    metrics_best_thresh_validate[model_name+fairness_method] = utils.describe_metrics(metrics_sweep[model_name+fairness_method])

    # We use the best threshold to obtain predicitions for test
    metrics_best_thresh_test[model_name+fairness_method] = utils.compute_metrics_postprocessing(
        dataset_true=test,
        dataset_preds=test_preds,
        model=methods[model_name+fairness_method], 
        threshold=metrics_best_thresh_validate[model_name+fairness_method]['best_threshold'], 
        scores_or_labels='labels'
    )




def PosprocEqoddsSCORES(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model_name, quality):
    # Global variables
    global methods
    global metrics_best_thresh_validate

     # Assign the correct name
    fairness_method = '_eqOdds'

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # Copy the model's predictions
    train_preds = utils.update_dataset_from_model(train, methods[model_name])
    val_preds = utils.update_dataset_from_model(val, methods[model_name])
    test_preds = utils.update_dataset_from_model(test, methods[model_name])

    # Assign the correct name
    model_name_metric = model_name + fairness_method + '_' + quality
    
    # Initialize the model 
    methods[model_name_metric] = CalibratedEqOddsPostprocessing(
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups,
        cost_constraint=quality,
        seed=seed)
    
    # Train the model
    methods[model_name_metric] = methods[model_name_metric].fit(train, train_preds)

    # Evaluate the model for a range of thresholds
    metrics_sweep[model_name_metric] = utils.metrics_postprocessing_threshold_sweep(
        dataset_true = val,
        dataset_preds = val_preds,
        model = methods[model_name_metric],
        thresh_arr = thresh_sweep,
        scores_or_labels = 'scores'
    )

    # Evaluate in best thresh
    metrics_best_thresh_validate[model_name_metric] = utils.describe_metrics(metrics_sweep[model_name_metric])

    # Using the best thresh, evaluate in test
    metrics_best_thresh_test[model_name_metric] = utils.compute_metrics_postprocessing(
        dataset_true=test,
        dataset_preds=test_preds,
        model=methods[model_name_metric], 
        threshold=metrics_best_thresh_validate[model_name_metric]['best_threshold'], 
        scores_or_labels='scores'
    )



def PosprocReject(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model_name, key_metric):
    # Global variables
    global methods
    global metrics_best_thresh_validate
    global fair_metrics_optrej

    # Assign the correct name
    fairness_method = '_RejOpt'
    model_name_metric = model_name + fairness_method + '_' + key_metric

    # Copy the datasets
    train, val, test = data_train.copy(deepcopy=True), data_val.copy(deepcopy=True), data_test.copy(deepcopy=True)

    # Copy predictions
    train_preds = utils.update_dataset_from_model(train, methods[model_name])
    val_preds = utils.update_dataset_from_model(val, methods[model_name])
    test_preds = utils.update_dataset_from_model(test, methods[model_name])

    # Train the model
    methods[model_name_metric] = RejectOptionClassification(
        unprivileged_groups=unprivileged_groups, 
        privileged_groups=privileged_groups, 
        metric_name=fair_metrics_optrej[key_metric],
        metric_lb=-0.01,
        metric_ub=0.01
        )

    # Train the model
    methods[model_name_metric] = methods[model_name_metric].fit(train, train_preds)
    
    # Obtain best threshold in val
    metrics_best_thresh_validate[model_name_metric] = utils.compute_metrics_postprocessing(
        dataset_true=val, 
        dataset_preds=val_preds, 
        model=methods[model_name_metric], 
        required_threshold=False)
    
    # Obtain it in test
    metrics_best_thresh_test[model_name_metric] = utils.compute_metrics_postprocessing(
        dataset_true=test, 
        dataset_preds=test_preds, 
        model=methods[model_name_metric], 
        required_threshold=False)

# Model training

## Grids

In [11]:
# DI remover
repair_level = 0.5                      
dir_grid = {                           
    'repair_level': [0.25, 0.5, 0.75]
}                


# MetaFair classifier
quality_constraints_meta = ['sr', 'fdr']
tau = 0.8   
metafair_grid = {
    'tau': [0.1, 0.2, 0.5, 0.8, 0.9]
}

# Prejudice index regularizer
pir_grid = {
    'eta': [ 0.5, 5.0, 50.0, 500.0]
}

# Adversarial learning
pir_grid = {
    'eta': [ 0.5, 5.0, 50.0, 500.0]
}

# Equal odds
# Quality constraints
quality_constraints_eqodds = ["weighted", 'fnr', 'fpr']

# Option rejection
# Fairness metrics
fair_metrics_optrej = {
    'spd': "Statistical parity difference",
    'aod': "Average odds difference",
    'eod': "Equal opportunity difference"
}

## Training

In [12]:
nvar = 2
 
measurement = 'combination'
combination = ['bal_acc', 'separation']
 
data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups, data_val_single, data_test_single = GermanDataset2V()
 
modelsNames = [
    'logreg',
    'xgboost',
    'adversarial',
    'metafair',
    'pir'
]


modelsBenchmark = [
    'logreg',
    'xgboost'
]

modelsFair = [
    'adversarial',
    'metafair_sr',
    'metafair_fdr',
    'pir'
]

modelsPre = [
    prefix + '_' + model_name for prefix in ['RW', 'DI'] for model_name in modelsBenchmark
]


modelsPost = modelsPre + modelsFair


modelsTrain = {
    'logreg': LogisticRegression,
    'xgboost': XGBClassifier,
    'adversarial': AdversarialDebiasing,
    'metafair': MetaFairClassifier,
    'pir': PrejudiceRemover
}

modelsArgs = {
    'logreg': {
        'solver': 'liblinear',
        'random_state': seed
    },
    'xgboost': {
        'eval_metric': 'error',
        'eta':0.1,
        'max_depth':6,
        'subsample':0.8
    },
    'adversarial': {
        'privileged_groups': privileged_groups,
        'unprivileged_groups': unprivileged_groups,
        'scope_name': 'debiased_classifier',
        'debias': True,
        'num_epochs': 80
    },
    'metafair': {
        'tau': 0.8,
        'sensitive_attr': sensitive_attribute,
        'type': 'sr',
        'seed': seed
    },
#    'metafair_fdr': {
#        'tau': 0.8,
#        'sensitive_attribute': sensitive_attribute,
#        'type': 'fdr',
#        'seed': seed
#    },
    'pir': {
        'sensitive_attr': sensitive_attribute,
        'eta': 50.0
    }
}

# Initialize dicts
methods = dict()

# Range of thresholds to evaluate our models
thresh_sweep = np.linspace(0.01, 1.0, 50)

metrics_sweep = dict()

# Store results from validation and test
metrics_best_thresh_validate = dict()
metrics_best_thresh_test = dict()

# Benchmarks
BenchmarkLogistic(data_train, data_val, data_test)
BenchmarkXGB(data_train, data_val, data_test)

# Pre processing + In processing
for model in modelsNames:
    if model == 'adversarial':
        tf.compat.v1.reset_default_graph()
        sess = tf.compat.v1.Session()
    PreprocRW(data_train, data_val, data_test, privileged_groups, unprivileged_groups, model, do_results = True)

    if model == 'adversarial':
        sess.close()
        tf.compat.v1.reset_default_graph()
        sess = tf.compat.v1.Session()

    PreprocDI(data_train, data_val, data_test, sensitive_attribute, repair_level, model, do_results = True)
    
    if model == 'adversarial':
        sess.close()


# Pre/In processing + Post processing

for quality in quality_constraints_meta:
    InprocMeta(data_train, data_val, data_test, sensitive_attribute, quality, tau = 0.8, do_results = True)
InprocPI(data_train, data_val, data_test, sensitive_attribute, eta = 50.0, do_results = True)

tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()
InprocAdvs(data_train, data_val, data_test, privileged_groups, unprivileged_groups, do_results = True)


for model in modelsPost:
    PosprocPlatt(data_train, data_val, data_test, privileged_groups, model)
    PosprocEqoddsLABELS(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model)
    for quality in quality_constraints_eqodds:
        PosprocEqoddsSCORES(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, quality)
    for key_metric in fair_metrics_optrej:
        PosprocReject(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, key_metric)
        
sess.close()

algorithm_performance_summary = pd.DataFrame(metrics_best_thresh_test).T
algorithm_performance_summary


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

epoch 0; iter: 0; batch classifier loss: 78.328941; batch adversarial loss: 0.603363
epoch 1; iter: 0; batch classifier loss: 65.281548; batch adversarial loss: 0.541025
epoch 2; iter: 0; batch classifier loss: 32.843430; batch adversarial loss: 0.521141
epoch 3; iter: 0; batch classifier loss: 38.578865; batch adversarial loss: 0.482325
epoch 4; iter: 0; batch classifier loss: 36.352013; batch adversarial loss: 0.504410
epoch 5; iter: 0; batch classifier loss: 51.872528; batch adversarial loss: 0.501763
epoch 6; iter: 0; batch classifier loss: 50.874722; batch adversarial loss: 0.522069
epoch 7; iter: 0; batch classifier loss: 40.608467; batch adversarial loss: 0.498124
epoch 8; iter: 0; batch classifier loss: 43.048775; batch adversarial loss: 0.531246
epoch 9; iter: 0; batch classifier loss: 43.378441; batch adversarial loss: 0.467275
epoch 10; iter: 0; batch classifi

,best_threshold,bal_acc,acc,independence,separation,sufficiency,auc
logreg,0.777755,0.711053,0.653333,0.212779,0.179526,0.260606,0.745201
xgboost,0.757551,0.699333,0.680000,0.109181,0.045250,0.136364,0.756112
RW_logreg,0.191837,0.541119,0.693333,0.009926,0.000893,0.160339,0.748465
DI_logreg,0.838367,0.666801,0.586667,0.255583,0.254205,0.601449,0.749444
RW_xgboost,0.131224,0.551323,0.700000,0.028536,0.040774,0.137006,0.736582
...,...,...,...,...,...,...,...
pir_eqOdds_fnr,0.676735,0.668418,0.666667,0.318519,0.200653,0.322785,0.733279
pir_eqOdds_fpr,0.777755,0.721257,0.660000,0.444444,0.236175,0.916667,0.733279
pir_RejOpt_spd,0.534646,0.652960,0.660000,0.103704,0.228591,0.328947,0.733279
pir_RejOpt_aod,0.574242,0.678319,0.680000,0.037037,0.091110,0.206169,0.733279


In [13]:
nvar = 1
 
measurement = 'combination'
combination = ['bal_acc', 'separation']
 
data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups = GermanDataset1V()
 
modelsNames = [
    'logreg',
    'xgboost',
    'adversarial',
    'metafair',
    'pir'
]


modelsBenchmark = [
    'logreg',
    'xgboost'
]

modelsFair = [
    'adversarial',
    'metafair',
    'pir'
]

modelsPre = [
    prefix + '_' + model_name for prefix in ['RW', 'DI'] for model_name in modelsBenchmark
]

modelsPost = modelsPre + modelsFair


modelsTrain = {
    'logreg': LogisticRegression,
    'xgboost': XGBClassifier,
    'adversarial': AdversarialDebiasing,
    'metafair': MetaFairClassifier,
    'pir': PrejudiceRemover
}

modelsArgs = {
    'logreg': {
        'solver': 'liblinear',
        'random_state': seed
    },
    'xgboost': {
        'eval_metric': 'error',
        'eta':0.1,
        'max_depth':6,
        'subsample':0.8
    },
    'adversarial': {
        'privileged_groups': privileged_groups,
        'unprivileged_groups': unprivileged_groups,
        'scope_name': 'debiased_classifier',
        'debias': True,
        'num_epochs': 80
    },
    'metafair': {
        'tau': 0.8,
        'sensitive_attr': sensitive_attribute,
        'type': 'sr',
        'seed': seed
    },
#    'metafair_fdr': {
#        'tau': 0.8,
#        'sensitive_attribute': sensitive_attribute,
#        'type': 'fdr',
#        'seed': seed
#    },
    'pir': {
        'sensitive_attr': sensitive_attribute,
        'eta': 50.0
    }
}

# Initialize dicts
methods = dict()

# Range of thresholds to evaluate our models
thresh_sweep = np.linspace(0.01, 1.0, 50)

metrics_sweep = dict()

# Store results from validation and test
metrics_best_thresh_validate = dict()
metrics_best_thresh_test = dict()

# Benchmarks
BenchmarkLogistic(data_train, data_val, data_test)
BenchmarkXGB(data_train, data_val, data_test)

# Pre processing + In processing
for model in modelsNames:
    PreprocRW(data_train, data_val, data_test, privileged_groups, unprivileged_groups, model, do_results = True)
    PreprocDI(data_train, data_val, data_test, sensitive_attribute, repair_level, model, do_results = True)

# Pre/In processing + Post processing

for quality in quality_constraints_meta:
    InprocMeta(data_train, data_val, data_test, sensitive_attribute, quality, tau = 0.8, do_results = True)
InprocPI(data_train, data_val, data_test, sensitive_attribute, eta = 50.0, do_results = True)
InprocAdvs(data_train, data_val, data_test, privileged_groups, unprivileged_groups, do_results = True)


for model in modelsPost:
    PosprocPlatt(data_train, data_val, data_test, privileged_groups, model)
    PosprocEqoddsLABELS(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model)
    for quality in quality_constraints_eqodds:
        PosprocEqoddsSCORES(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, quality)
    for key_metric in fair_metrics_optrej:
        PosprocReject(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, key_metric)


algorithm_performance_summary = pd.DataFrame(metrics_best_thresh_test).T
algorithm_performance_summary

epoch 0; iter: 0; batch classifier loss: 78.328941; batch adversarial loss: 0.583292
epoch 1; iter: 0; batch classifier loss: 65.280716; batch adversarial loss: 0.564662
epoch 2; iter: 0; batch classifier loss: 32.841087; batch adversarial loss: 0.517795
epoch 3; iter: 0; batch classifier loss: 38.843472; batch adversarial loss: 0.526234
epoch 4; iter: 0; batch classifier loss: 36.136360; batch adversarial loss: 0.543333
epoch 5; iter: 0; batch classifier loss: 51.419430; batch adversarial loss: 0.534661
epoch 6; iter: 0; batch classifier loss: 50.930008; batch adversarial loss: 0.572119
epoch 7; iter: 0; batch classifier loss: 40.479343; batch adversarial loss: 0.534250
epoch 8; iter: 0; batch classifier loss: 43.165482; batch adversarial loss: 0.562091
epoch 9; iter: 0; batch classifier loss: 42.964333; batch adversarial loss: 0.521938
epoch 10; iter: 0; batch classifier loss: 42.900841; batch adversarial loss: 0.545499
epoch 11; iter: 0; batch classifier loss: 38.518967; batch adver

ValueError: Variable debiased_classifier/classifier_model/W1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?

In [ ]:
algorithm_performance_summary = pd.DataFrame(metrics_best_thresh_test).T
algorithm_performance_summary

,best_threshold,bal_acc,acc,independence,separation,sufficiency,auc
logreg,0.777755,0.711053,0.653333,0.212779,0.179526,0.260606,0.745201
xgboost,0.757551,0.699333,0.680000,0.109181,0.045250,0.136364,0.756112
RW_logreg,0.717143,0.661063,0.658679,0.156145,0.135985,0.219985,0.743514
DI_logreg,0.818163,0.676702,0.600000,0.178660,0.182776,0.334783,0.744191
RW_xgboost,0.878776,0.659640,0.622095,0.241415,0.037121,0.011749,0.736582
DI_xgboost,0.656531,0.703374,0.706667,0.096774,0.155300,0.249480,0.757325
RW_adversarial,0.090816,0.506168,0.357673,0.188780,0.009091,0.406250,0.527233
DI_adversarial,0.838367,0.542029,0.673333,0.057072,0.131335,0.232143,0.467367
RW_metafair,0.636327,0.663103,0.674318,0.191546,0.169192,0.266712,0.692216
DI_metafair,0.636327,0.676399,0.606667,0.156328,0.139296,0.250000,0.728026


In [ ]:
nvar = 2

modelsNames = ['logreg', 'xgboost']

# Initialize dicts
methods = dict()

# Range of thresholds to evaluate our models
thresh_sweep = np.linspace(0.01, 1.0, 50)

metrics_sweep = dict()

# Store results from validation and test
metrics_best_thresh_validate = dict()
metrics_best_thresh_test = dict()

data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups, data_val_single, data_test_single = GermanDataset2V()

# Benchmarks
BenchmarkLogistic(data_train, data_val, data_test)
BenchmarkXGB(data_train, data_val, data_test)

# Pre processing
for model in modelsNames:
    PreprocRW(data_train, data_val, data_test, privileged_groups, unprivileged_groups, model, do_results = True)
    PreprocDI(data_train, data_val, data_test, sensitive_attribute, repair_level, model, do_results = True)

for quality in quality_constraints_meta:
    InprocMeta(data_train, data_val, data_test, sensitive_attribute, quality, tau = 0.8, do_results = True)
InprocPI(data_train, data_val, data_test, sensitive_attribute, eta = 50.0, do_results = True)
InprocAdvs(data_train, data_val, data_test, privileged_groups, unprivileged_groups, do_results = True)


# Post processing
for model in modelsNames:
    PosprocPlatt(data_train, data_val, data_test, privileged_groups, model)
    PosprocEqoddsLABELS(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model)
    for quality in quality_constraints_eqodds:
        PosprocEqoddsSCORES(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, quality)
    for key_metric in fair_metrics_optrej:
        PosprocReject(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, key_metric)

algorithm_performance_summary = pd.DataFrame(metrics_best_thresh_test).T
algorithm_performance_summary

epoch 0; iter: 0; batch classifier loss: 78.328941; batch adversarial loss: 0.603363
epoch 1; iter: 0; batch classifier loss: 65.281548; batch adversarial loss: 0.541025
epoch 2; iter: 0; batch classifier loss: 32.843430; batch adversarial loss: 0.521141
epoch 3; iter: 0; batch classifier loss: 38.578865; batch adversarial loss: 0.482325
epoch 4; iter: 0; batch classifier loss: 36.352013; batch adversarial loss: 0.504410
epoch 5; iter: 0; batch classifier loss: 51.872528; batch adversarial loss: 0.501763
epoch 6; iter: 0; batch classifier loss: 50.874722; batch adversarial loss: 0.522069
epoch 7; iter: 0; batch classifier loss: 40.608467; batch adversarial loss: 0.498124
epoch 8; iter: 0; batch classifier loss: 43.048775; batch adversarial loss: 0.531246
epoch 9; iter: 0; batch classifier loss: 43.378441; batch adversarial loss: 0.467275
epoch 10; iter: 0; batch classifier loss: 42.046997; batch adversarial loss: 0.495165
epoch 11; iter: 0; batch classifier loss: 37.851143; batch adver

,best_threshold,bal_acc,acc,independence,separation,sufficiency,auc
logreg,0.777755,0.711053,0.653333,0.212779,0.179526,0.260606,0.745201
xgboost,0.757551,0.699333,0.680000,0.109181,0.045250,0.136364,0.756112
RW_logreg,0.191837,0.541119,0.693333,0.009926,0.000893,0.160339,0.748465
DI_logreg,0.838367,0.666801,0.586667,0.255583,0.254205,0.601449,0.749444
RW_xgboost,0.131224,0.551323,0.700000,0.028536,0.040774,0.137006,0.736582
DI_xgboost,0.676735,0.729036,0.720000,0.157568,0.079733,0.147727,0.769852
metafair_sr,0.474694,0.718529,0.720000,0.329404,0.091943,0.098101,0.746818
metafair_fdr,0.656531,0.661245,0.593333,0.156328,0.120035,0.229167,0.736917
pir,0.818163,0.691554,0.620000,0.251852,0.068164,0.096154,0.733279
adversarial,0.474694,0.517882,0.626667,0.022222,0.080069,0.202703,0.440291


In [ ]:
nvar = 1

# Initialize dicts
methods = dict()

# Range of thresholds to evaluate our models
thresh_sweep = np.linspace(0.01, 1.0, 50)

metrics_sweep = dict()

# Store results from validation and test
metrics_best_thresh_validate = dict()
metrics_best_thresh_test = dict()

data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups = GermanDataset1V()

# Benchmarks
BenchmarkLogistic(data_train, data_val, data_test)
BenchmarkXGB(data_train, data_val, data_test)

# Pre processing
for model in modelsNames:
    PreprocRW(data_train, data_val, data_test, privileged_groups, unprivileged_groups, model, do_results = True)
    PreprocDI(data_train, data_val, data_test, sensitive_attribute, repair_level, model, do_results = True)

for quality in quality_constraints_meta:
    InprocMeta(data_train, data_val, data_test, sensitive_attribute, quality, tau = 0.8, do_results = True)
InprocPI(data_train, data_val, data_test, sensitive_attribute, eta = 50.0, do_results = True)
InprocAdvs(data_train, data_val, data_test, privileged_groups, unprivileged_groups, do_results = True)


# Post processing
for model in modelsNames:
    PosprocPlatt(data_train, data_val, data_test, privileged_groups, model)
    PosprocEqoddsLABELS(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model)
    for quality in quality_constraints_eqodds:
        PosprocEqoddsSCORES(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, quality)
    for key_metric in fair_metrics_optrej:
        PosprocReject(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, key_metric)

algorithm_performance_summary = pd.DataFrame(metrics_best_thresh_test).T
algorithm_performance_summary

epoch 0; iter: 0; batch classifier loss: 78.328941; batch adversarial loss: 0.583292
epoch 1; iter: 0; batch classifier loss: 65.280716; batch adversarial loss: 0.564662
epoch 2; iter: 0; batch classifier loss: 32.841087; batch adversarial loss: 0.517795
epoch 3; iter: 0; batch classifier loss: 38.843472; batch adversarial loss: 0.526234
epoch 4; iter: 0; batch classifier loss: 36.136360; batch adversarial loss: 0.543333
epoch 5; iter: 0; batch classifier loss: 51.419430; batch adversarial loss: 0.534661
epoch 6; iter: 0; batch classifier loss: 50.930008; batch adversarial loss: 0.572119
epoch 7; iter: 0; batch classifier loss: 40.479343; batch adversarial loss: 0.534250
epoch 8; iter: 0; batch classifier loss: 43.165482; batch adversarial loss: 0.562091
epoch 9; iter: 0; batch classifier loss: 42.964333; batch adversarial loss: 0.521938
epoch 10; iter: 0; batch classifier loss: 42.900841; batch adversarial loss: 0.545499
epoch 11; iter: 0; batch classifier loss: 38.518967; batch adver

,best_threshold,bal_acc,acc,independence,separation,sufficiency,auc
logreg,0.777755,0.711053,0.653333,0.212779,0.179526,0.260606,0.745201
xgboost,0.757551,0.699333,0.680000,0.109181,0.045250,0.136364,0.756112
RW_logreg,0.717143,0.661063,0.658679,0.156145,0.135985,0.219985,0.743514
DI_logreg,0.818163,0.676702,0.600000,0.178660,0.182776,0.334783,0.744191
RW_xgboost,0.878776,0.659640,0.622095,0.241415,0.037121,0.011749,0.736582
DI_xgboost,0.656531,0.703374,0.706667,0.096774,0.155300,0.249480,0.757325
metafair_sr,0.575714,0.616791,0.646667,0.218983,0.192961,0.298108,0.653869
metafair_fdr,0.636327,0.710750,0.660000,0.143921,0.094583,0.160714,0.728430
pir,0.797959,0.686603,0.613333,0.287841,0.277193,0.606667,0.722772
adversarial,0.070612,0.509598,0.346667,0.006203,0.031986,0.250000,0.586583


In [ ]:
modelsNames

['logreg', 'xgboost']

In [ ]:
data_test.labels.shape

(150, 1)

In [ ]:
nvar = 1

modelsNames = [
    'logreg',
    'xgboost',
    'adversarial',
    'metafair',
    'piremover'
]

modelsTrain = {
    'logreg': LogisticRegression,
    'xgboost': XGBClassifier,
    'adversarial': AdversarialDebiasing,
    'metafair': MetaFairClassifier,
    'piremover': PrejudiceRemover
}

modelsArgs = {
    'logreg': {
        'solver': 'liblinear',
        'random_state': seed
    },
    'xgboost': {
        'eval_metric': 'error',
        'eta':0.1,
        'max_depth':6,
        'subsample':0.8
    },
    'adversarial': {
        'privileged_groups': privileged_groups,
        'unprivileged_groups': unprivileged_groups,
        'scope_name': 'debiased_classifier',
        'debias': True,
        'sess': tf.Session(), # Mirar esto de la sesion
        'num_epochs': 80
    },
    'metafair': {
        'tau': 0.8,
        'sensitive_attribute': sensitive_attribute,
        'type': 'sr',
        'seed': seed
    },
#    'metafair_fdr': {
#        'tau': 0.8,
#        'sensitive_attribute': sensitive_attribute,
#        'type': 'fdr',
#        'seed': seed
#    },
    'piremover': {
        'sensitive_attr': sensitive_attribute,
        'eta': 50.0
    }
}

# Initialize dicts
methods = dict()

# Range of thresholds to evaluate our models
thresh_sweep = np.linspace(0.01, 1.0, 50)

metrics_sweep = dict()

# Store results from validation and test
metrics_best_thresh_validate = dict()
metrics_best_thresh_test = dict()

data_train, data_val, data_test, sensitive_attribute, privileged_groups, unprivileged_groups, data_val_single, data_test_single = GermanDataset2V()

# Benchmarks
BenchmarkLogistic(data_train, data_val, data_test)
BenchmarkXGB(data_train, data_val, data_test)

# Pre processing
for model in modelsNames:
    PreprocRW(data_train, data_val, data_test, privileged_groups, unprivileged_groups, model, do_results = True)
    PreprocDI(data_train, data_val, data_test, sensitive_attribute, repair_level, model, do_results = True)

#for quality in quality_constraints_meta:
#    InprocMeta(data_train, data_val, data_test, sensitive_attribute, quality, tau = 0.8, do_results = True)
## InprocPI(data_train, data_val, data_test, sensitive_attribute, eta = 50.0, do_results = True)
#InprocAdvs(data_train, data_val, data_test, privileged_groups, unprivileged_groups, do_results = True)
#
#
## Post processing
#for model in modelsNames:
#    PosprocPlatt(data_train, data_val, data_test, privileged_groups, model)
#    PosprocEqoddsLABELS(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model)
#    for quality in quality_constraints_eqodds:
#        PosprocEqoddsSCORES(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, quality)
#    for key_metric in fair_metrics_optrej:
#        PosprocReject(data_train, data_val, data_test, unprivileged_groups, privileged_groups, model, key_metric)

algorithm_performance_summary = pd.DataFrame(metrics_best_thresh_test).T
algorithm_performance_summary

ValueError: 'age' is not in list

# Hyper parameter tuning

In [ ]:

grids = {
    'RW': {
        
    },
    'DIR': {
        'repair_level': [0.25, 0.5, 0.75]
    },
    'MetaFair': {
        'tau': [0.1, 0.2, 0.5, 0.8, 0.9]
    },
    'PrejReg': {
        'eta': [ 0.5, 5.0, 50.0, 500.0]
    }

}

In [ ]:
utils.metrics_threshold_sweep_mult